In [38]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import uniform
from sympy import *
from scipy.stats import rv_continuous
import scipy

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go


pd.options.display.float_format = '{:,.4f}'.format

import warnings
warnings.filterwarnings("ignore")

from typing import Tuple

%matplotlib inline

# Задача 1


В течение месяца выборочно осуществлялась проверка торговых точек города по продаже овощей. Результаты двух проверок по недовесам покупателям одного вида овощей приведены в таблице:

|Интервалы|0-10|10-20|20-30|30-40|40-50|50-60|60-70|70-80|80-90|
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| $n_1$ | 3 | 10 | 15 | 20 | 12 | 5 | 25 | 15 | 5 |
| $n_2$ | 5 | 12 | 8 | 25 | 10 | 8 | 20 | 7 | 5 |


Можно ли считать при уровне значимости $0,05$, что недовесы овощей являются
устойчивым и закономерным процессом при продаже овощей в данном городе (т.е.
описываются одной и той же функцией распределения)? 

In [14]:
df = pd.DataFrame([[3, 10, 15, 20, 12, 5, 25, 15, 5],
                   [5, 12, 8, 25, 10, 8, 20, 7, 5]], 
                  columns=['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90'],
                  index=['n_1', 'n_2']).T

df

,n_1,n_2
0-10,3,5
10-20,10,12
20-30,15,8
30-40,20,25
40-50,12,10
50-60,5,8
60-70,25,20
70-80,15,7
80-90,5,5


## Расчет вручную

#### Считаем функции распределения

In [23]:
n = df['n_1'].sum()
m = df['n_2'].sum()

df['n_cumsum'] = df['n_1'].cumsum()
df['m_cumsum'] = df['n_2'].cumsum()

df['F_n'] = df['n_cumsum']/n
df['F_m'] = df['m_cumsum']/m

#### Считаем разницу функций распределения и выбираем максимум разницы

In [29]:
df['diff'] = abs(df['F_n'] - df['F_m'])

max_val = df['diff'].max()

#### Считаем статистику

In [52]:
t = float(sqrt(m*n/(m+n)) * max_val)
t

0.6447926610605738

In [48]:
t_critical = scipy.special.kolmogi(0.05)
t_critical

1.3580986393225507

In [54]:
p_value = scipy.special.kolmogorov(t)
p_value

0.19996947526025532

<div class="alert alert-block alert-info">
<b>Вывод:</b> Статистика меньше критического значения. Нулевую гипотезу отвергнуть не можем
</div>

## Рассчет с помощью библиотеке stats

In [72]:
stats.ks_2samp(df['n_1'], df['n_2'], alternative='greater', method='asymp')

KstestResult(statistic=0.1111111111111111, pvalue=0.8007374029168081)

# Задача 2

Вычислить статистику Колмогорова-Смирнова о гомогенности распределения по двум выборкам

|Возраст|21-22|23-24|25-26|27-28|29-30|31-32|33-34|35-36|37-38|39-40|
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| men | 4 | 11 | 5 | 7 | 0 | 5 | 9 | 13 | 20 | 6 |
| women | 7 | 4 | 1 | 11 | 12 | 4 | 2 | 4 | 8 | 9 |

In [80]:
df = pd.DataFrame([[4, 11, 5, 7, 0, 5, 9, 13, 20, 6],
                   [7, 4, 1, 11, 12, 4, 2, 4, 8, 9]], 
                  columns=['21-22', '23-24', '25-26', '27-28', '29-30', '31-32', '33-34', '35-36', '37-38', '39-40'],
                  index=['men', 'woman']).T

df

,men,woman
21-22,4,7
23-24,11,4
25-26,5,1
27-28,7,11
29-30,0,12
31-32,5,4
33-34,9,2
35-36,13,4
37-38,20,8
39-40,6,9


## Расчет вручную 

#### Считаем функции распределения

In [81]:
n = df['men'].sum()
m = df['woman'].sum()

df['n_cumsum'] = df['men'].cumsum()
df['m_cumsum'] = df['woman'].cumsum()

df['F_n'] = df['n_cumsum']/n
df['F_m'] = df['m_cumsum']/m

df

,men,woman,n_cumsum,m_cumsum,F_n,F_m
21-22,4,7,4,7,0.0500,0.1129
23-24,11,4,15,11,0.1875,0.1774
25-26,5,1,20,12,0.2500,0.1935
27-28,7,11,27,23,0.3375,0.3710
29-30,0,12,27,35,0.3375,0.5645
31-32,5,4,32,39,0.4000,0.6290
33-34,9,2,41,41,0.5125,0.6613
35-36,13,4,54,45,0.6750,0.7258
37-38,20,8,74,53,0.9250,0.8548
39-40,6,9,80,62,1.0000,1.0000


#### Считаем разницу функций распределения и выбираем максимум разницы

In [82]:
df['diff'] = abs(df['F_n'] - df['F_m'])

max_val = df['diff'].max()

#### Считаем статистику

In [84]:
t = float(sqrt(m*n/(m+n)) * max_val)
t

1.3536092732085314

In [85]:
t_critical = scipy.special.kolmogi(0.05)
t_critical

1.3580986393225507

In [97]:
p_value = scipy.special.kolmogorov(t)
p_value

0.051232265574690405

<div class="alert alert-block alert-info">
<b>Вывод:</b> Статистика меньше критического значения. Нулевую гипотезу отвергнуть не можем
</div>

## Рассчет с помощью библиотеке stats

In [107]:
stats.ks_2samp(df['men'], df['woman'])

KstestResult(statistic=0.3, pvalue=0.7869297884777761)